<img align="right" width="250" height="150" src="https://lh3.googleusercontent.com/p/AF1QipPWZQfa087JiVjutpUTVEGRnh6W214Wjm439gKQ=w1080-h608-p-no-v0">

## **Proyecto:** Retención de usuarios de la Plataforma Digital

#### El objetivo de este notebook es realizar el análisis EDA del archivo de `..\Detalle_Trx_Monetarias_YYYYMM.csv` con el objeto de ver que datos incorporar en el archivo de `..\Crosseling_ampliado_YYYYMM.csv`.

In [3]:
import pandas as pd
import pandas_profiling
import os

DIR = os.path.abspath(r'..\Data\Export') + '\\'

In [4]:
uc = [
    'TIPO_CANAL',
    'CANAL',
    'GRUPO_OPERACION',
    'TIPO_OPERACION',
    'TIPO_OPERACION2',
    'IMPORTE',
    'ORIGEN',
    'CANAL_RATIO',
    'CLASIF_TRX'
]

tc = dict(zip(uc, [str] * len(uc)))
tc.update({'IMPORTE': float})

In [5]:
file = DIR + 'Detalle_Trx_Monetarias_202204.csv'
df = pd.read_csv(file, index_col=False, usecols=uc, dtype=tc).sample(200_000) #Tomo una muestra sino pandas_profiling explota

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 200000 entries, 240479 to 2259006
Data columns (total 9 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   TIPO_CANAL       200000 non-null  object 
 1   CANAL            200000 non-null  object 
 2   GRUPO_OPERACION  200000 non-null  object 
 3   TIPO_OPERACION   200000 non-null  object 
 4   TIPO_OPERACION2  14355 non-null   object 
 5   IMPORTE          200000 non-null  float64
 6   ORIGEN           200000 non-null  object 
 7   CANAL_RATIO      200000 non-null  object 
 8   CLASIF_TRX       200000 non-null  object 
dtypes: float64(1), object(8)
memory usage: 15.3+ MB


In [7]:
# Analizamos
df.reset_index(drop=True, inplace=True)
report = pandas_profiling.ProfileReport(df)
report

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

#### Variables categóricas con menor grado de correlación *CANAL_RATIO* y *GRUPO_OPERACION*

In [8]:
# Remuevo los outliers del 1er y 3er quantil de Importe a ver que pasa
Q1 = df['IMPORTE'].quantile(0.25)
Q3 = df['IMPORTE'].quantile(0.75)
IQR = Q3 - Q1
print(Q1, Q3, IQR)

df = df[~((df['IMPORTE'] < (Q1 - 1.5 * IQR)) | (df['IMPORTE'] > (Q3 + 1.5 * IQR)))]
df.shape

1000.0 7100.0 6100.0


(179098, 9)

In [9]:
df.sample(20)

,TIPO_CANAL,CANAL,GRUPO_OPERACION,TIPO_OPERACION,TIPO_OPERACION2,IMPORTE,ORIGEN,CANAL_RATIO,CLASIF_TRX
198782,PRESENCIAL,TD (COMPRAS PEI),TESORERIA,TRANSFERENCIAS,NaN,1700.40,LINK-E,RESTO,TRANSFERENCIAS
104709,PRESENCIAL,TD (COMPRAS POSNET),TESORERIA,COMPRAS,NaN,3370.00,LINK-E,RESTO,EXCLUIDAS
24138,INTERNET,TD (COMPRAS E-COM),TESORERIA,COMPRAS,NaN,350.00,LINK-E,DIGITAL,EXCLUIDAS
166193,INTERNET,TD (COMPRAS E-COM),TESORERIA,COMPRAS,NaN,519.00,LINK-E,DIGITAL,EXCLUIDAS
91405,PRESENCIAL,MPOS,TESORERIA,COMPRAS,NaN,3670.00,LINK-E,RESTO,EXCLUIDAS
136013,PRESENCIAL,TD (COMPRAS POSNET),TESORERIA,COMPRAS,NaN,1023.50,LINK-E,RESTO,EXCLUIDAS
124509,PRESENCIAL,TD (COMPRAS POSNET),TESORERIA,COMPRAS,NaN,390.00,LINK-E,RESTO,EXCLUIDAS
117461,PRESENCIAL,TD (COMPRAS POSNET),TESORERIA,COMPRAS,NaN,949.70,LINK-E,RESTO,EXCLUIDAS
52005,PRESENCIAL,TD (COMPRAS POSNET),TESORERIA,COMPRAS,NaN,642.00,LINK-E,RESTO,EXCLUIDAS
103407,TERMINALES AUTOM SUC,ATM PROPIO,TESORERIA,EXTRACCIONES,NaN,1300.00,LINK-E,RESTO,EXCLUIDAS


In [10]:
# Analizamos
df.reset_index(drop=True, inplace=True)
report = pandas_profiling.ProfileReport(df)
report

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]